Wczytanie modułów


In [256]:
import lightgbm as lgb
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import json
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE

Wczytanie danych


In [257]:
data_csv = pd.read_csv('data.csv')
#data = data_csv.sample(n=len(data_csv))
data = data_csv
data['bmi'].fillna(data['bmi'].mean(),inplace=True)


<ipython-input-257-f421bf3ba1a8>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['bmi'].fillna(data['bmi'].mean(),inplace=True)


Przygotowanie danych

In [258]:
col = ["gender", "ever_married", "work_type", "Residence_type", "bmi", "smoking_status"]

data[col] = data[col].astype("category")


data.info()


numerical_features = ["age","hypertension","heart_disease", "avg_glucose_level"]
categorical_features = ["gender", "ever_married", "work_type", "Residence_type", "bmi", "smoking_status"]
target = "stroke"

X = data[numerical_features + categorical_features]
y = data[target]





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 5110 non-null   int64   
 1   gender             5110 non-null   category
 2   age                5110 non-null   float64 
 3   hypertension       5110 non-null   int64   
 4   heart_disease      5110 non-null   int64   
 5   ever_married       5110 non-null   category
 6   work_type          5110 non-null   category
 7   Residence_type     5110 non-null   category
 8   avg_glucose_level  5110 non-null   float64 
 9   bmi                5110 non-null   category
 10  smoking_status     5110 non-null   category
 11  stroke             5110 non-null   int64   
dtypes: category(6), float64(2), int64(4)
memory usage: 294.8 KB


Trenowanie modelu

In [259]:
pd.options.mode.chained_assignment = None
encoder = sklearn.preprocessing.OrdinalEncoder()


X[categorical_features] = encoder.fit_transform(X[categorical_features])
smote = SMOTE(random_state = 10)
X1, Y1 = smote.fit_resample(X, y)
Y1 = pd.DataFrame(Y1)
X1 = pd.DataFrame(X1)
print(Y1.value_counts)


X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=200)

classifier = lgb.LGBMClassifier()

# Trenowanie modelu na danych treningowych
classifier.fit(X_train, y_train)


<bound method DataFrame.value_counts of       stroke
0          1
1          1
2          1
3          1
4          1
...      ...
9717       1
9718       1
9719       1
9720       1
9721       1

[9722 rows x 1 columns]>
[LightGBM] [Info] Number of positive: 3905, number of negative: 3872
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001409 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2044
[LightGBM] [Info] Number of data points in the train set: 7777, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502122 -> initscore=0.008487
[LightGBM] [Info] Start training from score 0.008487


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


LGBMClassifier()

In [260]:
y_train.value_counts()

,count
stroke,
1,3905
0,3872


In [261]:
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred)

0.9838187702265372